In [1]:
from reachy_sdk import ReachySDK
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode
import time
import numpy as np

In [2]:
reachy = ReachySDK('localhost')

In [3]:
reachy.turn_off_smoothly('reachy')

In [ ]:
import os
path = '/home/reachy/repos/TicTacToe/tictactoe/images/cubes'
files = os.listdir(path)


for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index), '.jpg'])))

In [ ]:
class Robot():
    def __init__(self) -> None:
        self.reachy = ReachySDK('localhost')
    
    def setup(self):
        self.reachy.turn_on('head')
        self.reachy.head.look_at(x=1, y=0, z=0, duration=1.5) 
        self.reachy.head.l_antenna.speed_limit = 50.0
        self.reachy.head.r_antenna.speed_limit = 50.0
        self.reachy.head.l_antenna.goal_position = 0
        self.reachy.head.r_antenna.goal_position = 0
        #self.goto_rest_position()
    def play_pawn(self, grab_index, box_index):
        self.reachy.r_arm.r_gripper.speed_limit = 80
        self.reachy.r_arm.r_gripper.compliant = False
        self.reachy.turn_on('r_arm')

        # Goto base position
        self.goto_base_position() 
        self.reachy.r_arm.r_gripper.goal_position = -10 #open the gripper 
        path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/grab_{grab_index}.npz'
        self.goto_position(path)
        time.sleep(1)
        self.reachy.r_arm.r_gripper.compliant = False 
        self.reachy.r_arm.r_gripper.goal_position = 13 #close the gripper to take the cylinder 
        time.sleep(1)


        if grab_index >= 4:
            goto(
            goal_positions = {
                self.reachy.r_arm.r_shoulder_pitch : self.reachy.r_arm.r_shoulder_pitch.goal_position+10, 
                self.reachy.r_arm.r_elbow_pitch : self.reachy.r_arm.r_elbow_pitch.goal_position+10, 
                },
                duration =1.0,
                interpolation_mode=InterpolationMode.MINIMUM_JERK
            )

        # Lift it
        path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/lift.npz'
        self.goto_position(path)
        time.sleep(0.1)

        # Put it in box_index

        path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/put_{box_index}.npz'
        self.trajectoryPlayer(path)
        time.sleep(1)
        self.reachy.r_arm.r_gripper.compliant = False
        self.reachy.r_arm.r_gripper.goal_position = -10
        time.sleep(1)

        path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_{box_index}_upright.npz'
        self.goto_position(path)


        if box_index in (8, 9):

            path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_to_back.npz'
            self.goto_position(path)

        path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_rest.npz'
        self.goto_position(path)

        self.goto_rest_position()

    def goto_rest_position(self, duration=2.0):
        time.sleep(0.1)
        self.reachy.head.look_at(x=1, y=0, z=0, duration=1) 
        self.goto_base_position(0.6 * duration)
        time.sleep(0.1)

        self.reachy.turn_on('r_arm')

    def goto_base_position(self, duration=2.0):
 
        self.reachy.turn_on('r_arm')

        time.sleep(0.1)
        goto(
            goal_positions=
                    {self.reachy.r_arm.r_shoulder_pitch: 60,
                    self.reachy.r_arm.r_shoulder_roll: -15,
                    self.reachy.r_arm.r_arm_yaw: 0,
                    self.reachy.r_arm.r_elbow_pitch: -95,
                    self.reachy.r_arm.r_forearm_yaw: -15,
                    self.reachy.r_arm.r_wrist_pitch: -50,
                    self.reachy.r_arm.r_wrist_roll: 0},
                duration=1.0,
                interpolation_mode=InterpolationMode.MINIMUM_JERK
            )
        time.sleep(0.1)
        self.reachy.r_arm.r_shoulder_pitch.torque_limit = 75
        self.reachy.r_arm.r_elbow_pitch.torque_limit = 75

    def goto_position(self, path): 
        self.reachy.turn_on('r_arm')
        move = np.load(path)
        move.allow_pickle=1
        listMoves = move['move'].tolist()
        listTraj = {}
        for key,val in listMoves.items():
            listTraj[eval('self.'+key)] = float(val)
        goto(
            goal_positions=listTraj, 
            duration=2.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

    def trajectoryPlayer(self,path):
        self.reachy.turn_on('r_arm')
        move = np.load(path)
        move.allow_pickle=1
        #print(list(move.keys()))
        #print(move['move'].shape)
        listMoves = move['move'].tolist()
        #print(move['move'])
        #print(listMoves)
        listTraj = [val for key,val in listMoves.items()]
        listTraj = np.array(listTraj).T.tolist()

        sampling_frequency = 100  #en hertz

        recorded_joints = []
        for joint,val in listMoves.items():
            if 'neck' in joint : 
                fullName = 'self.' + joint
            elif 'r_' in joint: 
                fullName = 'self.' + joint
            elif 'l_' in joint: 
                fullName = 'self.' + joint
            recorded_joints.append(eval(fullName))
            
        first_point = dict(zip(recorded_joints, listTraj[0]))
        goto(first_point, duration=3.0)

        for joints_positions in listTraj:
            print(joints_positions)
            for joint, pos in zip(recorded_joints, joints_positions):
                joint.goal_position = pos
            time.sleep(1 / sampling_frequency)

    def rest(self, duration=1.0):
        self.reachy.turn_on('r_arm')
        time.sleep(0.1)
        goto(
            goal_positions=
                    {self.reachy.r_arm.r_shoulder_pitch: 55,
                    self.reachy.r_arm.r_shoulder_roll: -15,
                    self.reachy.r_arm.r_arm_yaw: 0,
                    self.reachy.r_arm.r_elbow_pitch: -85,
                    self.reachy.r_arm.r_forearm_yaw: -10,
                    self.reachy.r_arm.r_wrist_pitch: -50,
                    self.reachy.r_arm.r_wrist_roll: 0},
                duration=1.0,
                interpolation_mode=InterpolationMode.MINIMUM_JERK
            )
        time.sleep(0.1)
        self.reachy.r_arm.r_shoulder_pitch.torque_limit = 75
        self.reachy.r_arm.r_elbow_pitch.torque_limit = 75



In [ ]:
robot = Robot()

In [ ]:
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/put_1.npz'
robot.trajectoryPlayer(path)

In [ ]:
robot.reachy.turn_on('r_arm')
# Goto base position
robot.goto_base_position() 
robot.reachy.r_arm.r_gripper.goal_position = -10 #open the gripper 
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/grab_1.npz'
robot.goto_position(path)
time.sleep(1)
robot.reachy.r_arm.r_gripper.compliant = False 
robot.reachy.r_arm.r_gripper.goal_position = 13 #close the gripper to take the cylinder 
time.sleep(1)

In [ ]:
# Lift it
path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/lift.npz'
robot.goto_position(path)
time.sleep(0.1)



In [ ]:
# Put it in box_index

path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/put_1.npz'
robot.trajectoryPlayer(path)
time.sleep(4)
robot.reachy.r_arm.r_gripper.compliant = False
robot.reachy.r_arm.r_gripper.goal_position = -10
time.sleep(1)



In [ ]:
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_1_upright.npz'
robot.goto_position(path)


#if box_index in (8, 9):

#    path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_to_back.npz'
#    robot.goto_position(path)

path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/back_rest.npz'
robot.goto_position(path)

robot.goto_rest_position()

In [ ]:
robot.play_pawn(1,1)

In [ ]:
robot.reachy.turn_off_smoothly('r_arm')

In [ ]:
path = f'/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_nemo/put_9.npz'
trajectoryPlayer(path)

In [ ]:
for f in reachy.fans.values():
    f.on()

In [ ]:
robot.reachy.turn_off_smoothly('r_arm')



In [ ]:
for i in range(9):
    state = board.get_board()
    state_m  = np.where(state == 1, 3, state)
    state_m  = np.where(state_m == 2, 1, state_m)
    state_m  = np.where(state_m == 3, 2, state_m)
    game = state_m[::-1]
    game = np.array_split(state_m,3)
    print(game[0])
    print(game[1])
    print(game[2])
    print('     ')
    print('     ')

    action = - board.choose_next_action(state_m)[0] +9
    #board.play_pawn(1,action)
    time.sleep(2)


In [ ]:
def trajectoryPlayer(path):
    reachy.turn_on('r_arm')
    move = np.load(path)
    move.allow_pickle=1
    #print(list(move.keys()))
    #print(move['move'].shape)
    listMoves = move['move'].tolist()
    #print(move['move'])
    #print(listMoves)
    listTraj = [val for key,val in listMoves.items()]
    listTraj = np.array(listTraj).T.tolist()

    sampling_frequency = 100  #en hertz

    recorded_joints = []
    for joint,val in listMoves.items():
        if 'neck' in joint : 
            fullName = joint
        elif 'r_' in joint: 
            fullName = joint
        elif 'l_' in joint: 
            fullName = joint
        recorded_joints.append(eval(fullName))
        
    first_point = dict(zip(recorded_joints, listTraj[0]))
    goto(first_point, duration=3.0)

    for joints_positions in listTraj:
        print(joints_positions)
        for joint, pos in zip(recorded_joints, joints_positions):
            joint.goal_position = pos
        time.sleep(1 / sampling_frequency)

In [ ]:
path = '/home/reachy/repos/TicTacToe/tictactoe/movements/moves-2021_right/put_1.npz'

trajectoryPlayer(path)

In [ ]:
reachy.turn_off_smoothly('r_arm')